In [104]:
import cv2
import easyocr
import numpy as np
import os
import time


In [105]:

# Create the 'images' folder if it doesn't exist
output_folder = 'images'
car_regions_folder = 'car-regions'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if not os.path.exists(car_regions_folder):
    os.makedirs(car_regions_folder)

In [106]:
def preprocess_image(image):
    """Preprocess the image for better OCR."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    return gray

In [107]:

def detect_number_plate(image):
    """Detect number plate text using EasyOCR."""
    reader = easyocr.Reader(['en'])  # Initialize EasyOCR reader for English
    results = reader.readtext(image)

    # Extract text
    plate_text = " ".join([res[1] for res in results])
    return plate_text

In [108]:
def process_frame(frame):
    """
    Process a single frame to detect number plate text.
    :param frame: A video frame or image.
    :return: Extracted number plate text and the processed frame with annotations.
    """
    # Preprocess the frame
    preprocessed_frame = preprocess_image(frame)

    # Detect number plate text
    text = detect_number_plate(preprocessed_frame)

    # Annotate the frame with detected text (optional)
    if text.strip():  # Only annotate if text is detected
        cv2.putText(frame, text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    return text, frame

In [109]:
def is_valid_plate(text):
    """Validate the detected text as a potential license plate."""
    # Check if the text length and characters are reasonable for a license plate
    return 5 <= len(text) <= 12 and any(char.isdigit() for char in text)

In [110]:
def process_video_with_number_plate(video_path):
    """
    Process a video to detect cars and extract number plate text.
    :param video_path: Path to the video file.
    """
    print("Initializing video capture...")
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Unable to open video source.")
        return

    print("Video source opened successfully. Press 'q' to quit.")

    frame_counter = 0  # Counter to track frames

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("End of video or unable to read frame.")
            break

        frame_counter += 1

        # Process every 90th frame (approximately every 3 seconds for 30 FPS video)
        if frame_counter % 90 == 0:
            print(f"Processing frame {frame_counter}...")

            # Process the frame for number plate recognition
            text, annotated_frame = process_frame(frame)

            if text.strip():  # Only log if text is detected
                print(f"Detected Text: {text}")

            # Save the preprocessed frame and results (optional)
            preprocessed_path = f"images/frame_{frame_counter}.jpg"
            cv2.imwrite(preprocessed_path, annotated_frame)
            print(f"Frame saved to: {preprocessed_path}")

        # Display the video feed with annotations
        cv2.imshow("Video Feed", frame)

        # Exit on 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting video capture...")
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
def capture_and_process_image():
    """Capture an image from live feed and extract number plate text."""
    print("Initializing camera...")

    cap = cv2.VideoCapture(0, cv2.CAP_AVFOUNDATION)

    if not cap.isOpened():
        print("Error: Unable to access the camera")
        return

    print("Press 'c' to capture an image and extract number plate text or 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to read from the camera")
            break

        cv2.imshow("Camera Feed", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            image_path = "captured_image.jpg"
            cv2.imwrite(image_path, frame)

            # Preprocess image and extract text
            print("Processing image to extract number plate text...")
            preprocessed_image = preprocess_image(frame)
            text = detect_number_plate(preprocessed_image)
            print(f"Extracted Text: {text}")
            break
        elif key == ord('q'):
            print("Exiting without capturing.")
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    # Choose a video or live feed for testing
    video_path = '/Users/mohan/Downloads/Automatic Number Plate Recognition (ANPR) _ Vehicle Number Plate Recognition (1).mp4'
    if video_path == "0":
        capture_and_process_image()  # Use the live feed function
    else:
        process_video_with_number_plate(video_path)

Initializing video capture...
Video source opened successfully. Press 'q' to quit.
Processing frame 90...
Detected Text: IP Camera 2017-08-29 05:33: Number Plate SL SAFEPRO
Frame saved to: images/frame_90.jpg
Processing frame 180...
Detected Text: IP Camera 2017-08-21 05:33: Number Plate KA 02 MN 1826 Aedn SAFEPRO
Frame saved to: images/frame_180.jpg
Processing frame 270...
Detected Text: IP Camera 201y-08-21 05.33: Number Plate FpNE 2e SAF_PRO
Frame saved to: images/frame_270.jpg
Processing frame 360...
Detected Text: IP Camera 017-08-2 05*44: Number Plate Sa SAE: PRO X021h7256
Frame saved to: images/frame_360.jpg
Processing frame 450...
Detected Text: IP Camera Z01y-08-2} 05*44: Number Plate KA 02 MM 9091 SL( SAE PRO X1024H7258
Frame saved to: images/frame_450.jpg
Processing frame 540...
Detected Text: IP Camera ZQ17-08-23 05*45: Number Plate KA 02 MM 9091 Sa KA 02 Hm909t SAE PRO X1020H7258
Frame saved to: images/frame_540.jpg
Processing frame 630...
Detected Text: IP Camera 017-08-2

: 